DoppleGANger (model training done on Colab)

In [ ]:
%pip install ydata-synthetic==1.3.1

In [2]:
#Imports

import numpy as np
import pandas as pd

#for ydata-synthetic
from ydata_synthetic.synthesizers.timeseries import TimeSeriesSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters


In [4]:
#loading the data
real_data = pd.read_csv("../Data/GOOG.csv")

num_cols = ['Open', 'High', 'Low', 'Adj Close', '7ma', '14ma', '21ma',
       '7atr', '14atr', '21atr', '7upper', '7lower', '14upper', '14lower',
       '21upper', '21lower', 'MSFT Close', 'AMZN Close', 'META Close', 'AAPL Close', 'Close']
cat_cols = ['Date']

#dropping 7 rows to get length as multiple of sequence length (required by the model)
n = 7
real_data.drop(real_data.tail(n).index,inplace = True)

In [5]:
#defining the model parameters
model_args = ModelParameters(batch_size=120,
                             lr=0.001,
                             betas=(0.2, 0.9),
                             latent_dim=20,
                             gp_lambda=2,
                             pac=1)

train_args = TrainParameters(epochs=400, sequence_length=30,
                             sample_length=5, rounds=1,
                             measurement_cols=["Date"])

In [ ]:
#training the synthesizer
model_dop_gan = TimeSeriesSynthesizer(modelname='doppelganger', model_parameters=model_args)
model_dop_gan.fit(real_data, train_args, num_cols=num_cols, cat_cols=cat_cols)
model_dop_gan.save("DopGAN_synthesizer")

In [7]:
#generating synthetic data
dopgan_synth_data = model_dop_gan.sample(n_samples=1500)
synth_df = pd.concat(dopgan_synth_data, axis=0)

#formatting
synth_df = synth_df.dropna(subset=['Date']) #dropping Nan date rows
synth_df['Date'] = pd.to_datetime(synth_df['Date']) #setting date to datetime
synth_df = synth_df.sort_values(by='Date') #ordering by date

In [8]:
#saving updated version
synth_df.to_csv("../Data/DopGAN_synth_data.csv")
synth_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3095 entries, 0 to 4
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        3095 non-null   datetime64[ns]
 1   Open        3095 non-null   float64       
 2   High        3095 non-null   float64       
 3   Low         3095 non-null   float64       
 4   Adj Close   3095 non-null   float64       
 5   MSFT Close  3095 non-null   float64       
 6   AMZN Close  3095 non-null   float64       
 7   META Close  3095 non-null   float64       
 8   AAPL Close  3095 non-null   float64       
 9   7ma         3095 non-null   float64       
 10  14ma        3095 non-null   float64       
 11  21ma        3095 non-null   float64       
 12  7atr        3095 non-null   float64       
 13  14atr       3095 non-null   float64       
 14  21atr       3095 non-null   float64       
 15  7upper      3095 non-null   float64       
 16  7lower      3095 non-null   floa